In [26]:
import requests
import pandas as pd
import defs
from datetime import datetime, timedelta
import pytz
import defs
import plotly.graph_objects as go 

Debo obtener la data de ESTA semana. Parametros en RFC3339.  
Considerando que solo obtengo 5000 velas por llamada de la API y que cada dia tengo 1440 velasde M1, tengo que hacer 2.016 llamadas de la API por cada toda una semana.  
**NO HAY CONVERSION DE HORAS, LA HORA SOLO ES UNA Y VAMOS A USAR FORMATO Z UTC+0**  
- FORMATO RFC 3339:
    - 2024-07-03T03:08:00.000000000Z  (le resto 4 horas al UTC+0)


Como el API usa los valores Z UTC+0, voy a basar toda mi logica en este formato, bajo esa premisa mis horas importantes son:

- Inicio del dia de NY en formato Z: 04:00:00.000000000Z:
    - 2024-07-03T04:00:00.000000000Z
- Timeframe para Liquidity Grab NY 09:30-10:00 am:
    -   2024-07-03T13:30:00.000000000Z
- Silverbullet Start NY 10:00am:
    -   2024-07-03T14:00:00.000000000Z
- Silverbullet End NY 11:00am :
    -   2024-07-03T15:00:00.000000000Z
- Fin de sesion NY 17:00:
    -   2024-07-03T21:00:00.000000000Z

*NOTA:*  
Notar que en RFC339, la variable Z se usa para indicar que es UTC+0, por lo que la usare siempre (https://medium.easyread.co/understanding-about-rfc-3339-for-datetime-formatting-in-software-engineering-940aa5d5f68a).  


In [291]:
#Funcion para obtener hora actual de NY en formato Z UTC-0
import pytz
from datetime import datetime

def obtener_hora_nueva_york_rfc3339():
    # Definir la zona horaria de Nueva York
    zona_horaria_ny = pytz.timezone('America/New_York')

    # Obtener la fecha y hora actual en la zona horaria de Nueva York
    ahora_ny = datetime.now(zona_horaria_ny)

    # Formatear la fecha y hora en el formato deseado
    hora_formateada = ahora_ny.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    return hora_formateada

# Ejemplo de uso
hora_actual_ny = obtener_hora_nueva_york_rfc3339()
print("Hora actual en Nueva York:", hora_actual_ny)


Hora actual en Nueva York: 2024-07-04T00:55:16.686759Z


In [292]:
# Validacion de horario de verano en NY (retorno booleano)
import pytz
from datetime import datetime

def esta_en_horario_verano_nueva_york():
    # Definir la zona horaria de Nueva York
    zona_horaria_ny = pytz.timezone('America/New_York')

    # Obtener la fecha y hora actual en la zona horaria de Nueva York
    ahora_ny = datetime.now(zona_horaria_ny)

    # Verificar si estamos en horario de verano
    esta_en_horario_verano = ahora_ny.dst() != zona_horaria_ny.dst(None)

    return esta_en_horario_verano

esta_en_horario_verano_nueva_york()

True

In [293]:
# Hora 04:00 del dia de HOY

import pytz
from datetime import datetime, timedelta

def obtener_hora_04_00_utc_rfc3339():
    # Obtener la fecha y hora actual en UTC+0
    ahora_utc = datetime.now(pytz.utc)

    # Crear un nuevo objeto datetime con la hora fijada a las 04:00 del mismo día
    hora_04_00 = ahora_utc.replace(hour=4, minute=0, second=0, microsecond=0)

    # Formatear la nueva fecha y hora en formato RFC3339 con la 'Z'
    hora_rfc3339 = hora_04_00.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    return hora_rfc3339

# Ejemplo de uso
hora_04_00_utc_rfc3339 = obtener_hora_04_00_utc_rfc3339()
print("Hora 04:00 UTC+0 (RFC3339):", hora_04_00_utc_rfc3339)


Hora 04:00 UTC+0 (RFC3339): 2024-07-04T04:00:00.000000Z


In [294]:
# Obtengo la hora 04:00 de hace 7 dias, validando previamente si es horario de verano 
import pytz
from datetime import datetime, timedelta

def obtener_hora_04_00_utc_rfc3339_hace_7_dias():
    # Obtener la fecha y hora actual en UTC+0
    ahora_utc = datetime.now(pytz.utc)

    # Restar 7 días a la fecha y hora actual
    hace_7_dias = ahora_utc - timedelta(days=7)

    # Validar si es horario de verano
    horario_verano = esta_en_horario_verano_nueva_york()
    
    if horario_verano: 
        # Crear un nuevo objeto datetime con la hora fijada a las 04:00 del mismo día hace 7 días
        hora_ajustada = hace_7_dias.replace(hour=4, minute=0, second=0, microsecond=0)
    else:
        hora_ajustada = hace_7_dias.replace(hour=5, minute=0, second=0, microsecond=0)

    # Formatear la nueva fecha y hora en formato RFC3339 con la 'Z'
    hora_rfc3339 = hora_ajustada.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    return hora_rfc3339

# Ejemplo de uso
hora_04_00_utc_rfc3339_hace_7_dias = obtener_hora_04_00_utc_rfc3339_hace_7_dias()
print("Hora 04:00 UTC+0 (RFC3339) hace 7 días:", hora_04_00_utc_rfc3339_hace_7_dias)


Hora 04:00 UTC+0 (RFC3339) hace 7 días: 2024-06-27T04:00:00.000000Z


### Llamada al API y conversion a Dataframe

In [295]:
def get_candles (instrument, granularity, starting_date, count):
    session = requests.Session()
    url = f"{defs.OANDA_URL}/instruments/{instrument}/candles"
    params = {
        'granularity' : granularity,
        'price' : "MBA",
        'from' : starting_date,
        'count': count
    }
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    candle_data = response.json()
    return candle_data


def convert_to_df(candle_data):
    prices = ["mid", "bid", "ask"]
    ohlc = ["o", "h", "l", "c"]
    our_data = []

    for candle in candle_data["candles"]:
        if candle["complete"] == False:
            continue
        new_dict = {}
        new_dict["time"] = candle["time"]
        new_dict["volume"] = candle["volume"]
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame(our_data)



In [296]:
# PRUEBO LA FUNCION
instrument = "EUR_USD"
granularity = "M1"
starting_date = obtener_hora_04_00_utc_rfc3339_hace_7_dias()
ahoprita = '2024-07-04T03:12:25.253345Z'

initial_json = get_candles(instrument, granularity, ahoprita, 5000)
initial_df = convert_to_df(initial_json)
initial_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-07-04T03:12:00.000000000Z,25,1.07866,1.07872,1.07865,1.07868,1.07857,1.07864,1.07857,1.07860,1.07875,1.07879,1.07873,1.07875
1,2024-07-04T03:13:00.000000000Z,8,1.07866,1.07866,1.07866,1.07866,1.07858,1.07859,1.07858,1.07859,1.07873,1.07874,1.07873,1.07874
2,2024-07-04T03:14:00.000000000Z,1,1.07864,1.07864,1.07864,1.07864,1.07856,1.07856,1.07856,1.07856,1.07872,1.07872,1.07872,1.07872
3,2024-07-04T03:15:00.000000000Z,12,1.07864,1.07866,1.07864,1.07865,1.07857,1.07858,1.07856,1.07858,1.07872,1.07874,1.07871,1.07872
4,2024-07-04T03:16:00.000000000Z,8,1.07864,1.07864,1.07862,1.07864,1.07857,1.07857,1.07855,1.07857,1.07872,1.07872,1.07870,1.07871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2024-07-04T04:47:00.000000000Z,13,1.07850,1.07852,1.07850,1.07852,1.07842,1.07844,1.07842,1.07844,1.07859,1.07861,1.07859,1.07861
96,2024-07-04T04:48:00.000000000Z,38,1.07852,1.07854,1.07851,1.07852,1.07844,1.07845,1.07843,1.07844,1.07860,1.07862,1.07859,1.07860
97,2024-07-04T04:49:00.000000000Z,47,1.07852,1.07856,1.07852,1.07854,1.07844,1.07848,1.07844,1.07846,1.07861,1.07864,1.07860,1.07863
98,2024-07-04T04:50:00.000000000Z,26,1.07855,1.07858,1.07854,1.07858,1.07847,1.07850,1.07847,1.07850,1.07863,1.07866,1.07862,1.07865


### Logica de Iteracion para obtencion de datos consecutiva del API

In [311]:
# Intento hacer la llamada al API dentro de otra funcion que llame desde una semana atras 

hora_inicio = obtener_hora_04_00_utc_rfc3339_hace_7_dias()
instrument = "EUR_USD"
granularity = "M1"

def get_historical_candles(starting_date, instrument, granularity, count=5000, df_acumulado=None):

    json_data = get_candles(instrument, granularity, starting_date, 5000)
    print("se jalo data empezando en: ", starting_date)
    df = convert_to_df(json_data)
    last_candle = df.iloc[-1, 0]
    print ("la ultima vela es: ", last_candle)

    #condicion de cierre de recursividad
    if df.shape[0] == 1: 
        print("se cumplio la condicion de cierre del bucle recursivo")
        #df_no_duplicates = df_acumulado.drop_duplicates(subset=['time'])
        return df_acumulado.drop_duplicates(subset=['time']).reset_index(drop = True)

    # condicion para primera iteracion
    if df_acumulado is None: 
        print("se cumplio la condicion de inicio del bucle")
        df_acumulado = df
    else:
        print("se concatenan los df")
        df_acumulado = pd.concat([df_acumulado, df])
    

    print("===================================")
    print("se sigue ejecutando el bucle recursivo con el ultimo candle como nuevo limite")
    return get_historical_candles(last_candle, instrument, granularity, count=5000, df_acumulado=df_acumulado)




working_df = get_historical_candles(hora_inicio, instrument, granularity)
working_df

se jalo data empezando en:  2024-06-27T04:00:00.000000Z
la ultima vela es:  2024-07-02T15:57:00.000000000Z
se cumplio la condicion de inicio del bucle
se sigue ejecutando el bucle recursivo con el ultimo candle como nuevo limite
se jalo data empezando en:  2024-07-02T15:57:00.000000000Z
la ultima vela es:  2024-07-04T05:14:00.000000000Z
se concatenan los df
se sigue ejecutando el bucle recursivo con el ultimo candle como nuevo limite
se jalo data empezando en:  2024-07-04T05:14:00.000000000Z
la ultima vela es:  2024-07-04T05:14:00.000000000Z
se cumplio la condicion de cierre del bucle recursivo


,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-06-27T04:00:00.000000000Z,31,1.06906,1.06909,1.06906,1.06908,1.06899,1.06902,1.06899,1.06901,1.06913,1.06916,1.06913,1.06916
1,2024-06-27T04:01:00.000000000Z,8,1.06908,1.06909,1.06908,1.06908,1.06901,1.06902,1.06901,1.06901,1.06915,1.06916,1.06915,1.06915
2,2024-06-27T04:02:00.000000000Z,9,1.06906,1.06912,1.06906,1.06912,1.06899,1.06904,1.06899,1.06904,1.06914,1.06919,1.06914,1.06919
3,2024-06-27T04:03:00.000000000Z,11,1.06914,1.06918,1.06912,1.06918,1.06907,1.06911,1.06905,1.06911,1.06921,1.06925,1.06920,1.06925
4,2024-06-27T04:04:00.000000000Z,6,1.06918,1.06918,1.06916,1.06916,1.06911,1.06911,1.06909,1.06909,1.06926,1.06926,1.06923,1.06923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7189,2024-07-04T05:09:00.000000000Z,13,1.07856,1.07856,1.07855,1.07856,1.07849,1.07849,1.07848,1.07848,1.07864,1.07864,1.07862,1.07863
7190,2024-07-04T05:10:00.000000000Z,12,1.07852,1.07854,1.07852,1.07854,1.07844,1.07847,1.07844,1.07846,1.07861,1.07861,1.07860,1.07861
7191,2024-07-04T05:12:00.000000000Z,22,1.07854,1.07854,1.07852,1.07854,1.07847,1.07847,1.07845,1.07847,1.07861,1.07861,1.07860,1.07861
7192,2024-07-04T05:13:00.000000000Z,30,1.07854,1.07856,1.07852,1.07855,1.07846,1.07849,1.07845,1.07848,1.07861,1.07863,1.07860,1.07862


In [ ]:
# stress test NO CORRER
#choclo = get_historical_candles('2024-02-27T04:00:00.000000Z', instrument, granularity, count=5000, df_acumulado=None)
#choclo


## Grafic de las velas

In [32]:
hora_inicio = '2024-07-05T01:42:00.000000000Z'
instrument = "EUR_USD"
granularity = "M5"

df_to_plot = get_historical_candles(hora_inicio, instrument, granularity)


se jalo data empezando en:  2024-07-05T01:42:00.000000000Z
la ultima vela es:  2024-07-05T06:05:00.000000000Z
se cumplio la condicion de inicio del bucle
se sigue ejecutando el bucle recursivo con el ultimo candle como nuevo limite
se jalo data empezando en:  2024-07-05T06:05:00.000000000Z
la ultima vela es:  2024-07-05T06:05:00.000000000Z
se cumplio la condicion de cierre del bucle recursivo


In [33]:

def plot_df(df):
    fig = go.Figure()
    fig.add_trace(go.Candlestick(
        x=df.time, open=df.mid_o, high=df.mid_h, low=df.mid_l, close=df.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor = "#24A06B",
        decreasing_fillcolor = "#CC2E3C",
        increasing_line_color = "#2EC886",
        decreasing_line_color = "#FF3A4C"
    ))
    fig.update_layout(width=1000, height=400,
                    margin=dict(l=10, r=10, b=10, t=10),
                    font = dict(size=10, color="#e1e1e1"),
                    paper_bgcolor = "#1e1e1e",
                    plot_bgcolor =  "#1e1e1e",
                    )
    fig.update_xaxes(
        gridcolor = "#1f292f",
        showgrid=True, fixedrange=True, rangeslider=dict(visible=True)
    )
    fig.update_yaxes(
        gridcolor = "#1f292f",
        showgrid=True,
    )
    return fig

plot_fig = plot_df(df_to_plot)
plot_fig.show()

## Analisis de la estrategia

#### Definiciones previas
- Inicio del dia de NY en formato Z: 04:00:00.000000000Z:
    - 2024-07-03T04:00:00.000000000Z
- Timeframe para Liquidity Grab NY 09:30-10:00 am:
    -   2024-07-03T13:30:00.000000000Z
- Silverbullet Start NY 10:00am:
    -   2024-07-03T14:00:00.000000000Z
- Silverbullet End NY 11:00am :
    -   2024-07-03T15:00:00.000000000Z
- Fin de sesion NY 17:00:
    -   2024-07-03T21:00:00.000000000Z
#### Parametros de la estrategia:
- Para considerar una toma de liquidez, debo considerar la siguiente condición:
    - 
#### Criterios de la estrategia:
- Pre-Session:
    - Entre las 09:30 y 10:am NY, buscar una toma de liquidez.



# UTILIDADES A NECESITAR LUEGO

In [303]:
# Concatenar varios df. usar ignore_index = False para que no reescriba los indices

import pandas as pd

# Crear cinco DataFrames de ejemplo con índices personalizados
df1 = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6]
}, index=['a', 'b', 'c'])

df2 = pd.DataFrame({
    'A': [7, 8, 9],
    'B': [10, 11, 12]
}, index=['d', 'e', 'f'])

df3 = pd.DataFrame({
    'A': [13, 14, 15],
    'B': [16, 17, 18]
}, index=['g', 'h', 'i'])

df4 = pd.DataFrame({
    'A': [19, 20, 21],
    'B': [22, 23, 24]
}, index=['j', 'k', 'l'])

df5 = pd.DataFrame({
    'A': [25, 26, 27],
    'B': [28, 29, 30]
}, index=['m', 'n', 'o'])

# Anexar los cinco DataFrames
df_combined = pd.concat([df1, df2, df3, df4, df5])

print(df_combined)

    A   B
a   1   4
b   2   5
c   3   6
d   7  10
e   8  11
f   9  12
g  13  16
h  14  17
i  15  18
j  19  22
k  20  23
l  21  24
m  25  28
n  26  29
o  27  30
